In [18]:
import os
import sys
from importnb import Notebook
from typing import Optional, Union, List, Callable, Any
from io import BytesIO
import struct
import numpy as np
import pandas as pd

PKG_ROOT = os.path.dirname(os.path.realpath(os.getcwd()))
if not PKG_ROOT in sys.path:
    sys.path.append(PKG_ROOT)

In [19]:
with Notebook():
    from src.dtypes import *

In [20]:
OptimValues = {'Gold': 1000000
                ,'Str': 100
                ,'Dex': 100
                ,'Int': 100
                ,'Wis': 100
                ,'Con': 100
                ,'Cha': 100
                ,'Rank': 50}

Feats = pd.read_csv(os.path.join(PKG_ROOT, 'resources', 'feats.csv'), index_col=0)
Feats = Feats[Feats['Selected']==1]['FeatIndex'].tolist()



In [28]:
pd.set_option('display.max_rows', 200)

In [21]:
class Player:
    
    HeaderPattern = '4s4s12I'
    StructPattern = '3I'
    LabelPattern = '16s'
    FieldPattern = '2I'
    DwordPattern = 'I'
    HeaderSize = struct.calcsize(HeaderPattern)
    StructSize = struct.calcsize(StructPattern)
    LabelSize = struct.calcsize(LabelPattern)
    FieldSize = struct.calcsize(FieldPattern)
    DwordSize = struct.calcsize(DwordPattern)
    
    def __init__(self, filename:str):
        self.source = os.path.join(PKG_ROOT, 'saves/original', filename)
        self.save = os.path.join (PKG_ROOT, 'saves/modified', filename)
        with open(self.source, 'rb') as s:
            self.buffer = s.read()
        self.structs = []
        self.fields = []
        self.labels = []
        self.parse_header()
        self.unpack()
        
    def parse_header(self):
        (self.FileType
         ,self.Version
         ,self.structOffset
         ,self.structCount
         ,self.fieldOffset
         ,self.fieldCount
         ,self.labelOffset
         ,self.labelCount
         ,self.dataOffset
         ,self.dataSize
         ,self.indiceOffset
         ,self.indiceSize
         ,self.listOffset
         ,self.listSize) = struct.unpack(self.HeaderPattern, self.buffer[:self.HeaderSize])
        self.source = {'struct': BytesIO(self.buffer[self.structOffset: self.fieldOffset])
                       ,'field': BytesIO(self.buffer[self.fieldOffset: self.labelOffset])
                       ,'label': BytesIO(self.buffer[self.labelOffset: self.dataOffset])
                       ,'data': BytesIO(self.buffer[self.dataOffset: self.indiceOffset])
                       ,'indice': BytesIO(self.buffer[self.indiceOffset: self.listOffset])
                       ,'list': BytesIO(self.buffer[self.listOffset:])
                       }
        self.target = {k: b'' for k in self.source.keys()}

    def unpack(self):
        self.labels = [struct.unpack(self.LabelPattern, self.source['label'].read(self.LabelSize))[0].decode('latin').rstrip('\x00') for _ in range(self.labelCount)]
        i = -1
        self.structs = [NWStruct(i:=i+1).unpack(self.source) for _ in range(self.structCount)]
        i = -1
        self.fields = [NWField(i:=i+1).unpack(self.source, self.structs, self.labels) for _ in range(self.fieldCount)]
        [_.bind_fields(self.fields) for _ in self.structs]
        
    def pack(self, save:Optional[str]=None):
        self.target = {k: b'' for k in self.source.keys()}
        labels = list()
        pack = [_.pack(self.target, labels) for _ in self.structs]
        self.target['label'] = b''.join([struct.pack(self.LabelPattern, _.encode('latin')) for _ in labels])
        header = struct.pack("4s4s", self.FileType, self.Version)
        content = b''
        offset = struct.calcsize(self.HeaderPattern)
        for k, v in self.target.items():
            if k == 'struct':
                length = len(self.target[k]) // self.StructSize
            elif k == 'field':
                length = len(self.target[k]) // 12
            elif k == 'label':
                length = len(self.target[k]) // self.LabelSize
            else:
                length = len(v)
            header += struct.pack("2I", offset + len(content), length)
            content += v
        content = header + content
        if save is None:
            save = self.save
        with open(save, 'wb') as t:
            t.write(content)
            
    def field_summary(self):
        return pd.DataFrame([_.summary() for _ in self.fields])
    
    def change_simple_value(self, label:str, newValue:Any):
        df = self.field_summary()
        match = df[df['Label'] == label]
        if not df.empty:
            for f in match['Field']:
                f.content.value = newValue
                
      
    def add_to_list(self, label, newValues:List):
        '''
        Only works on fields of list that contains single field structs, such as featlist
        Using the first struct in the list as template
        '''
        df = self.field_summary()
        match = df[df['Label'] == label]
        structIndex = len(self.structs) - 1
        #fieldsIndex = len(self.fields) # new fields needs no index
        if not df.empty:
            for field, l in match[['Field', 'Value']].values:
                structTemplate = self.structs[l[0]]
                values = [self.structs[_].fields[0].content.value for _ in l]
                values = set(newValues).difference(values)
                newStructs = [structTemplate.like(structIndex:=structIndex+1, [v]) for v in values]
                self.structs.extend(newStructs)
                field.content.value = field.content.value + tuple(range(structIndex - len(values) + 1, structIndex + 1))
                
    def optimize_value(self):
        for k, v in OptimValues.items():
            self.change_simple_value(k, v)
        self.add_to_list('FeatList', Feats)
        self.pack()
                    
        

In [23]:
s1 = Player("earalebrielfren.bic")

In [25]:
s1.optimize_value()